In [64]:
import pandas
import requests #use requests over 'urllib.request'. Work-around for encoding problem. See: https://stackoverflow.com/questions/47419104/unicodedecodeerror-utf-8-codec-cant-decode-python3
import json
import geojson
import folium

In [70]:
#GDELT GeoJson Endpoint
url = "https://api.gdeltproject.org/api/v1/gkg_geojson?MAXROWS=1"

#Issue request to URL
request = requests.get(url)
print(request.status_code)
if request.status_code == 200: data = request.json()
print(type(data))
print("---------")
print(data['features'])

200
<class 'dict'>
---------
[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-66.0, 8.0]}, 'properties': {'urlpubtimedate': '2018-02-01T18:45:00Z', 'name': 'Venezuela', 'urltone': -6.21, 'url': 'http://minci.gob.ve/2018/02/comunicado-venezuela-repudio-acciones-terroristas-en-republica-de-mali/', 'mentionedthemes': ';EPU_POLICY_GOVERNMENT;TERROR;ARMEDCONFLICT;TERROR;ARMEDCONFLICT;TAX_ETHNICITY_VENEZUELAN;GENERAL_GOVERNMENT;GENERAL_GOVERNMENT;KILL;CRISISLEX_CRISISLEXREC;CRISISLEX_T02_INJURED;CRISISLEX_T03_DEAD;EPU_CATS_NATIONAL_SECURITY;TAX_ETHNICITY_MALI;TAX_ETHNICITY_MALI;'}}]


In [71]:
#I think this solution works!
#However, this will probably just work for my data only.

flat_dictionary = {}
for feature in data['features']:
    for key, value in feature.items():
      if type(feature[key]) == dict:
        for k, v in feature[key].items():
            if k in flat_dictionary.keys():
                flat_dictionary[k].append(feature[key][k])
            else:
                flat_dictionary[k] = [v]
   
df_features = pandas.DataFrame.from_dict(flat_dictionary)
df_features

,coordinates,mentionedthemes,name,type,url,urlpubtimedate,urltone
0,"[-66.0, 8.0]",;EPU_POLICY_GOVERNMENT;TERROR;ARMEDCONFLICT;TE...,Venezuela,Point,http://minci.gob.ve/2018/02/comunicado-venezue...,2018-02-01T18:45:00Z,-6.21


In [67]:
#reformat dataframe and expand coordinates into lat/lon columns
#use lambda function to iterate over coordinates column?

In [68]:
#another thing to try
#via https://www.haykranen.nl/2016/02/13/handling-complex-nested-dicts-in-python/

# class DictQuery(dict):
#     def get(self, path, default = None):
#         keys = path.split("/")
#         val = None

#         for key in keys:
#             if val:
#                 if isinstance(val, list):
#                     val = [ v.get(key, default) if v else None for v in val]
#                 else:
#                     val = val.get(key, default)
#             else:
#                 val = dict.get(self, key, default)

#             if not val:
#                 break;

#         return val

In [69]:
# from: https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json    
#a function that flattens dictionaries into lists
#I'm not sure how to implement this for my data...
# def dict_generator(indict, pre=None):
#     pre = pre[:] if pre else []
#     if isinstance(indict, dict):
#         for key, value in indict.items():
#             if isinstance(value, dict):
#                 for d in dict_generator(value, [key] + pre):
#                     yield d
#             elif isinstance(value, list) or isinstance(value, tuple):
#                 for v in value:
#                     for d in dict_generator(v, [key] + pre):
#                         yield d
#             else:
#                 yield pre + [key, value]
#     else:
#         yield indict
        
# for i in dict_generator(data):
#     print(i)